In [1]:
import os, torch

#model_id = "microsoft/Phi-3.5-mini-instruct"
model_id = "microsoft/Phi-3-mini-4k-instruct"

os.environ['CUDA_VISIBLE_DEVICES'] = str(5)

## Setup Dataset

In [2]:
import json 
f = open("../wikipedia.json", "r")
data = json.load(f)

articles = [art for art in data['data']]
print(articles[0].keys())
articles = [{"title": a['title'], "content": "\n".join([p['context'] for p in a['paragraphs']])} for a in articles]

dict_keys(['title', 'paragraphs'])


In [3]:
from langchain_core.documents import Document
docs = [Document(page_content=a['content'], metadata={"title":a['title']}) for a in articles]

In [4]:
documents = docs[:10]

## Setup Models

In [5]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_use_double_quant=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_compute_dtype=torch.bfloat16
        )
model = AutoModelForCausalLM.from_pretrained(
            model_id,
            torch_dtype="auto",
            trust_remote_code=True,
            #quantization_config=bnb_config
        ).to('cuda')

tokenizer = AutoTokenizer.from_pretrained(model_id)

from langchain_huggingface.llms import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

pipe = pipeline("text-generation",
                model=model,
                tokenizer=tokenizer, 
                device=0,
                max_new_tokens=1024)

llm = HuggingFacePipeline(pipeline=pipe)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

## Create RAGAS dataset from dataset

In [6]:
from ragas.testset.generator import TestsetGenerator
from ragas.testset.evolutions import simple, reasoning, multi_context
from langchain_huggingface import HuggingFaceEmbeddings

# documents = load your documents

# generator with openai models
#generator_llm = ChatOpenAI(model="gpt-3.5-turbo-16k")
#critic_llm = ChatOpenAI(model="gpt-4")
generator_llm = llm
critic_llm = llm
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

generator = TestsetGenerator.from_langchain(
    generator_llm,
    critic_llm,
    embeddings
)

# Change resulting question type distribution
distributions = {
    simple: 0.5,
    multi_context: 0.4,
    reasoning: 0.1
}

print(generator)


/cfs/home/u024236/Documents/Phi3.5/env_phi35/lib64/python3.11/site-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_name" in _VertexAIBase has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/cfs/home/u024236/Documents/Phi3.5/env_phi35/lib64/python3.11/site-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_name" in _VertexAICommon has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/cfs/home/u024236/Documents/Phi3.5/env_phi35/lib64/python3.11/site-packages/ragas/metrics/__init__.py:4: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydan

TestsetGenerator(generator_llm=LangchainLLMWrapper(run_config=RunConfig(timeout=180, max_retries=10, max_wait=60, max_workers=16, exception_types=(<class 'Exception'>,), log_tenacity=False, seed=42)), critic_llm=LangchainLLMWrapper(run_config=RunConfig(timeout=180, max_retries=10, max_wait=60, max_workers=16, exception_types=(<class 'Exception'>,), log_tenacity=False, seed=42)), embeddings=<ragas.embeddings.base.LangchainEmbeddingsWrapper object at 0x7ff0788e8690>, docstore=InMemoryDocumentStore(splitter=<langchain_text_splitters.base.TokenTextSplitter object at 0x7ff0788e80d0>, nodes=[], node_embeddings_list=[], node_map={}, run_config=RunConfig(timeout=180, max_retries=10, max_wait=60, max_workers=16, exception_types=(<class 'Exception'>,), log_tenacity=False, seed=42)))


In [7]:
# use generator.generate_with_llamaindex_docs if you use llama-index as document loader
testset = generator.generate_with_langchain_docs(documents, 2, distributions,raise_exceptions=False) 

embedding nodes: 0it [00:00, ?it/s]

ExceptionInRunner: The runner thread which was running the jobs raised an exeception. Read the traceback above to debug it. You can also pass `raise_exceptions=False` incase you want to show only a warning message instead.

Problem is in docstore the nodes appear to come with Embedding and Keyphrases but empty so they dont go into the executor to get embeddi

In [ ]:
test_df = testset.to_pandas()
test_df.head()

## Try with Langchain dataset

In [ ]:
from langchain_community.document_loaders import WikipediaLoader
documents = WikipediaLoader(query="Intelligence", load_max_docs=10).load()
len(documents)

# use generator.generate_with_llamaindex_docs if you use llama-index as document loader
testset = generator.generate_with_langchain_docs(documents, 2, distributions,raise_exceptions=False) 

## Try Llama-Index

In [ ]:
import json
from llama_index.core import Document

# Read the JSON file
f = open("../wikipedia.json", "r")
data = json.load(f)
articles = [art for art in data['data']]

# Print the keys of the first article (for debugging)
print(articles[0].keys())

# Process the articles
articles = [{"title": a['title'], "content": "\n".join([p['context'] for p in a['paragraphs']])} for a in articles]

# Create LlamaIndex Document objects
docs = [Document(text=a['content'], metadata={"title": a['title']}) for a in articles]

# use generator.generate_with_llamaindex_docs if you use llama-index as document loader
testset = generator.generate_with_langchain_docs(docs, 2, distributions,raise_exceptions=False) 